In [2]:
using ToQUBO

# LP Knapsack Problem

$$
\begin{array}{rl}
    \max & \mathbf{c}^{\small\mathbf{\text{T}}}\mathbf{x} \\
    \text{s.t.} & \mathbf{w}^{\small\mathbf{\text{T}}}\mathbf{x} \le C\\
    ~ & \mathbf{x}_i \in \{0, 1\}
\end{array}
$$

In [3]:
include("../test/models/knapsack.jl")

print(Knapsack.model)

LoadError: MethodError: no method matching _single_variable_flag(::MathOptInterface.ZeroOne)
[0mClosest candidates are:
[0m  _single_variable_flag([91m::Type{<:MathOptInterface.EqualTo}[39m) at C:\Users\pedroxavier\.julia\packages\MathOptInterface\jPhq9\src\Utilities\variables_container.jl:68
[0m  _single_variable_flag([91m::Type{<:MathOptInterface.GreaterThan}[39m) at C:\Users\pedroxavier\.julia\packages\MathOptInterface\jPhq9\src\Utilities\variables_container.jl:69
[0m  _single_variable_flag([91m::Type{<:MathOptInterface.LessThan}[39m) at C:\Users\pedroxavier\.julia\packages\MathOptInterface\jPhq9\src\Utilities\variables_container.jl:70
[0m  ...